GET/SET Tuya Zigbee IOT Devices

In [1]:
import hashlib
import hmac
import json
import urllib
import urllib.parse
from urllib.request import urlopen, Request
from datetime import datetime
import requests
from pandas import DataFrame


BASE_URL       = "https://openapi.tuyaeu.com"
LOGIN_URL      = "/v1.0/token?grant_type=1"
ATTRIBUTES_URL = "/v2.0/cloud/thing/{device_id}/shadow/properties"

client_id = "vtj3ft7hadfea5y3yvs7"
client_secret = "16347a3b50984ae696e6887e4d503fa2"

def make_request(url, params=None, headers=None):
    if params:
        url = url + "?" + urllib.parse.urlencode(params)
    request = Request(url, headers=headers or {})

    try:
        with urlopen(request, timeout=10) as response:
            return response, response.read().decode("utf-8")

    except Exception as error:
        return error, ""

def get_timestamp(now = datetime.now()):
    return str(int(datetime.timestamp(now)*1000))

def get_sign(payload, key):
    byte_key = bytes(key, 'UTF-8')
    message = payload.encode()
    sign = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return sign.upper()

def get_access_token(now):
    # now = datetime.now()

    timestamp = get_timestamp(now)
    string_to_sign = client_id + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        LOGIN_URL
    signed_string = get_sign(string_to_sign, client_secret)

    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + LOGIN_URL, headers = headers)

    json_result = json.loads(body)["result"]
    access_token = json_result["access_token"]
    return access_token

def get_status(device_id):
    access_token = get_access_token(datetime.now())
    url = ATTRIBUTES_URL.format(device_id=device_id)
    timestamp = get_timestamp()
    string_to_sign = client_id + access_token + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        url
    
    # print('aromax: ' + string_to_sign)
    signed_string = get_sign(string_to_sign, client_secret)
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + url, headers = headers)

    json_result = json.loads(body)
    properties = json_result["result"]["properties"]
    output = list()
    for item in properties:
        if ('value' in item) and ('code' in item):
            if (item['code'] == 'temp_set') or (item['code'] == 'temp_current') or (item['code'] == 'TempCurrent') or (item['code'] == 'TempSet') or (item['code'] == 'switch_1'):
                if (item['code'] == 'temp_current') or (item['code'] == 'TempSet') or (item['code'] == 'TempCurrent'):
                    output.append({item['code']: item['value'] / 10})
                elif (item['code'] == 'switch_1'):
                    if item['value'] == True:
                        output.append({'HeatPump': 'ON'})
                    else:
                        output.append({'HeatPump': 'OFF'})
                else:
                    output.append({item['code']: item['value']})
    return output

def set_temperature(device_id, temperature):
    # get_device_info = get_status(device_id)
    # is_thermostat = any('TempSet' in d for d in get_device_info)
    # if is_thermostat:
    #     temperature = temperature * 10
    access_token = get_access_token(datetime.now())
    temperature = str(temperature)
    sign_url = "/v1.0/devices/" + device_id + "/commands"
    body_string = '{ "commands": [ { "code": "temp_set", "value": ' + temperature + ' } ] }'
    body_string_encoded = hashlib.sha256(body_string.encode()).hexdigest()

    sign_string1 = "POST\n" + body_string_encoded + "\n\n" + sign_url
    
    timestamp = get_timestamp()

    sign_string2 = client_id + access_token + timestamp + sign_string1
    
    command_url = "https://openapi.tuyaeu.com/v1.0/devices/" + device_id + "/commands"

    signed_string = get_sign(sign_string2, client_secret)
    
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    data = {
        "commands": [
            {"code": "temp_set", "value": temperature}
        ]
    }

    response = requests.post(command_url, data=body_string, headers=headers)

    if response.status_code == 200:
        print('Temperature set successfully')
    else:
        print('Error setting temperature:', response.text)
        
def set_heatpump(device_id, command):
    access_token = get_access_token(datetime.now())
    sign_url = "/v1.0/devices/" + device_id + "/commands"
    body_string = '{ "commands": [ { "code": "switch_1", "value": ' + command + ' } ] }'
    body_string_encoded = hashlib.sha256(body_string.encode()).hexdigest()
    sign_string1 = "POST\n" + body_string_encoded + "\n\n" + sign_url
    
    timestamp = get_timestamp()

    sign_string2 = client_id + access_token + timestamp + sign_string1
    
    command_url = "https://openapi.tuyaeu.com/v1.0/devices/" + device_id + "/commands"

    signed_string = get_sign(sign_string2, client_secret)
    
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response = requests.post(command_url, data=body_string, headers=headers)
    if response.status_code == 200:
        print('Heatpump set Successfully')
    else:
        print('Error setting Heatpump:', response.text)

Listing of all the linked devices for each clients in a python "clients" list type variable

In [2]:
clients = [
    #Client -1: Saiful
    ["bfc6fdb8a9d07ef71cyuud", #For convention, we will set 1st item to be HUB of that house/client
     'bfc8da7daa7a40e10dgtgk',
     "bf7adcca218736d1beltu0"],
    #Client -2: Sas
    [
        "bf0867de0cbdec04744czw", #For convention, we will set 1st item to be HUB of that house/client
        "bf7356030aa2552ff4akq1",
        "bf247826822493b7052aya",
        "bfe22383bb69caf809simm"
    ]
    
]

Get Device Status Client-wise

In [3]:
for i in range(len(clients)):
    print(f"Client-{i + 1}")
    for j in range(len(clients[i])):
        if j > 0:
            print(get_status(clients[i][j]))

Client-1
[{'HeatPump': 'ON'}]
[{'temp_set': 24}, {'temp_current': 18.0}]
Client-2
[{'TempSet': 2.7}, {'TempCurrent': 5.3}]
[{'temp_set': 24}, {'temp_current': 11.0}]
[{'temp_set': 25}, {'temp_current': 13.0}]


Setting temperature for each client we got from the Survery form (all room temperature/global)

In [4]:
set_temp = [24, 25] # 24 degree celsius for Client-1 & 25 degree celsius for Client-2
for i in range(len(clients)):
    for j in range(len(clients[i])):
        if j > 0:
            print(get_status(clients[i][j]))
            set_temperature(clients[i][j], set_temp[i])

[{'temp_set': 24}, {'temp_current': 18.0}]
Temperature set successfully
[{'TempSet': 2.7}, {'TempCurrent': 5.3}]
Temperature set successfully
[{'temp_set': 24}, {'temp_current': 11.0}]
Temperature set successfully
[{'temp_set': 25}, {'temp_current': 13.0}]
Temperature set successfully


Setting a particular room's temperature to 26 degree celsius for Client-2

In [5]:
set_temperature(clients[1][2], 26)

Temperature set successfully


Turn on Heat pump for Client-1

In [6]:
set_heatpump(clients[0][1], 'true')

Heatpump set Successfully


Turn off Heat pump for Client-1

In [7]:
set_heatpump(clients[0][1], 'false')

Heatpump set Successfully
